In [1]:

from analysis.probability import cumulative_reduction_optimized
from evaluate import evaluate_single

probability_generator = cumulative_reduction_optimized
tau = 0.001

def eval_simple(note):
    return evaluate_single(probability_generator(note), tau=tau,root_note=note)

In [2]:
import numpy as np
from analysis.musical import consonance_ordered_notes, empirical_probabilities
from analysis.spike_tensor import generate_spike_tensor
from cache import get_spikes
from evaluate import predicted_consonance_scores, predicted_probabilities

root_note = "C4"
notes = consonance_ordered_notes(root_note)
notes_spikes = [get_spikes(note) for note in notes]
consonance_ordered_tensors = [generate_spike_tensor(spikes, tau=tau) for spikes in notes_spikes]

root_tensor = generate_spike_tensor(get_spikes(root_note), tau=tau)
probability_tensor = probability_generator(root_note)

def try_eval(tensors, metric, debug=False):
    scores = predicted_consonance_scores(probability_tensor, tensors, root_tensor)

    P_predicted = predicted_probabilities(scores)

    if debug:
        def print_arr(arr):
            print(np.round(np.array(arr), 3))
        print("Scores")
        print_arr(scores)
        print("P_predicted")
        print_arr(P_predicted)
        print("P_empirical")
        print_arr(empirical_probabilities())

    print(metric(P_predicted))

In [3]:
from doctest import debug
from evaluate import js_divergence


try_eval(consonance_ordered_tensors, js_divergence, debug=True)

Scores
[-15.783 -17.773 -17.825 -17.993 -18.001 -17.164 -17.701 -17.398 -17.375
 -16.647 -17.735 -17.539]
P_predicted
[0.313 0.032 0.025 0.001 0.    0.118 0.042 0.085 0.088 0.191 0.038 0.065]
P_empirical
[0.147 0.133 0.12  0.107 0.107 0.093 0.08  0.067 0.053 0.053 0.027 0.013]
0.3965227436078749


In [4]:
import numpy as np

def shuffle_attempts(metric, N=10):
    data = []
    for i in range(N):
        A = consonance_ordered_tensors[0][0][0]
        np.random.shuffle(consonance_ordered_tensors)
        B = consonance_ordered_tensors[0][0][0]
        if not np.array_equal(A, B):
            value = try_eval(consonance_ordered_tensors, metric)
            data.append(value)

In [5]:
from evaluate import js_divergence


shuffle_attempts(js_divergence)

0.39343818680536624
0.3287912589177932
0.3438795071251344
0.32299800889466834
0.4042231037835214
0.30842111553758833
0.4504984410785446
0.3653716826364866
0.4591172169691094


In [6]:
import os

from cache import SPIKES_DIR


root_note = "C4"
notes = consonance_ordered_notes(root_note)
notes = [notes[0], notes[-1]]
notes_spikes = [get_spikes(note) for note in notes]
tensors = [generate_spike_tensor(spikes, tau=tau) for spikes in notes_spikes]

white_noise_spikes = np.load(os.path.join(SPIKES_DIR, "white_noise.npy"))
tensors.append(generate_spike_tensor(white_noise_spikes, tau=tau))

root_tensor = generate_spike_tensor(get_spikes(root_note), tau=tau)

scores = predicted_consonance_scores(probability_tensor, tensors, root_tensor)
print("Scores")
print(np.round(np.array(scores)))

P_predicted = predicted_probabilities(scores)
print("P_predicted")
print(np.round(np.array(P_predicted), 2))

print("Most Cons, Least Cons, White Noise")


Scores
[-16. -18. -31.]
P_predicted
[0.53 0.47 0.  ]
Most Cons, Least Cons, White Noise


In [7]:
keys = ["C4", "C#4", "D4", "D#4", "E4", "F4", "F#4", "G4", "G#4", "A4", "A#4", "B4"]

for key in keys:
    print(f"{key}: {eval_simple(key)}")

C4: 60
C#4: 58
D4: 60
D#4: 63
E4: 65
F4: 58
F#4: 61
G4: 70
G#4: 69
A4: 71
A#4: 61
B4: 75


In [8]:
consonance_ordered_notes(root_note)

['C5', 'G4', 'F4', 'E4', 'D#4', 'A4', 'G#4', 'D4', 'C#4', 'B4', 'A#4', 'F#4']

In [9]:
# shuffle_attempts(kl_divergence_reverse)
# shuffle_attempts(kl_divergence)